In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import numpy as np
import pandas as pd
import glob
import os

In [2]:
#read in the human reference genome and parse it into a dictionary
record_dict = SeqIO.to_dict(SeqIO.parse('GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta', 'fasta'))

In [17]:
file_bank = glob.glob(os.path.join('read_gdc_download_20200619_175755.482406', '*.seg.v2.txt'))

In [18]:
final_results = pd.DataFrame()
#write a for loop to read in the files as a bed format
for file in file_bank:
    file_bed = pd.read_csv(file, sep='\t', header=0)
    
    #filter the csv frile to a pandas dataframe with these specified columns
    results1 = file_bed[['GDC_Aliquot', 'Chromosome', 'Start','End','Segment_Mean']]
    
     #filter segment length, copy gain, then find the sequence
    Length = results1['End'] - results1['Start']
    results1['Length'] = Length
    results2 = results1.loc[(results1['Length']>=10) & (results1['Length']<= 500000) & (results1['Segment_Mean'] >= 0.4)]
    
    sequences = []
    
    for index ,row in results2.iterrows():
        chromosome = 'chr' + str(row['Chromosome'])
        start = int(row['Start'])
        end = int(row['End'])
        chromosome_sequence = record_dict[chromosome].seq 
        sequences.append(str(chromosome_sequence[start:end]))
    
    #set sequence as a separate dataframe
    sequences1 = pd.DataFrame(sequences, columns=['Sequence'])
     
    #reset the index of the previous generated dataframe
    results2 = results2.reset_index()
    results2.drop("index", axis=1)
    
    #join the sequence dataframe to the original dataframe 
    results2 = results2.join(sequences1, how='right')

    final_results = final_results.append(results2)

In [19]:
final_results['Length'].isna().sum()

0

## Shape of Data

In [20]:
print(final_results.shape)

(2547, 8)


In [21]:
final_results.head

<bound method NDFrame.head of     index                           GDC_Aliquot Chromosome      Start  \
0       4  26a56799-544a-449b-8237-fc9b8196a804          2  241129553   
1      37  26a56799-544a-449b-8237-fc9b8196a804         19   47792055   
0       8  782af57b-6ac7-40a3-925c-880169d50340          6   24946443   
1      33  782af57b-6ac7-40a3-925c-880169d50340         15   27566806   
2      35  782af57b-6ac7-40a3-925c-880169d50340         15   40774677   
..    ...                                   ...        ...        ...   
11    195  b5939768-e706-43d4-9f3c-b454327bade8         20     472817   
12    204  b5939768-e706-43d4-9f3c-b454327bade8         20   36984415   
0     137  9046317c-b457-4702-9c00-536c520f1cef          6   16946114   
1     199  9046317c-b457-4702-9c00-536c520f1cef          8   34872345   
2     269  9046317c-b457-4702-9c00-536c520f1cef         10   87515763   

          End  Segment_Mean  Length  \
0   241129927        1.0309     374   
1    47898238  

In [22]:
#turn it into a csv file
final_results.to_csv('cnv_results_read.csv', sep='\t', index=False)

## Head of Data

In [23]:
pd.read_csv('cnv_results_read.csv', sep='\t', header=0).head()

,index,GDC_Aliquot,Chromosome,Start,End,Segment_Mean,Length,Sequence
0,4,26a56799-544a-449b-8237-fc9b8196a804,2,241129553,241129927,1.0309,374,TCCAATGCTTTTAGCATCAATTCTAAGAAAAATATTTCAGTGTTGG...
1,37,26a56799-544a-449b-8237-fc9b8196a804,19,47792055,47898238,0.5420,106183,ACAGAGTTTTAAAACATCAACTTTTAAAGATAGATGCTTTCTTTAC...
2,8,782af57b-6ac7-40a3-925c-880169d50340,6,24946443,24947577,1.0726,1134,TATCCTGTAATCCCAGCTACTCAGGAGGCTGAGTCAGGAGAGTCGC...
3,33,782af57b-6ac7-40a3-925c-880169d50340,15,27566806,27566865,1.1361,59,AAAGACATTTTAAATATTAAGACACAAATAGGACAAAAGCAAAAAA...
4,35,782af57b-6ac7-40a3-925c-880169d50340,15,40774677,40774860,1.6100,183,ATTTCATTGCATAGCGTCCTTGCTAGTTGAACGTGGTGAAAAGGAC...
